In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install tree
!pip install -qU wandb
!pip install -qU bbox-utility

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (38.7 kB/s)
Selecting previously unselected package tree.
(Reading database ... 157604 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 1.8 MB 2.2 MB/s 
     |████████████████████████████████| 158 kB 27.8 MB/s 
     |███████████████████████

# Импортировать необходимые библиотеки

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys

from joblib import Parallel, delayed

from IPython.display import display

# Настройки W&B

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login()
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


# Meta Data

In [ ]:
!tree /content/drive/MyDrive/hackathon/datasets --filelimit 100

/content/drive/MyDrive/hackathon/datasets
├── images [1904 entries exceeds filelimit, not opening dir]
├── labels [1904 entries exceeds filelimit, not opening dir]
├── masks [1904 entries exceeds filelimit, not opening dir]
├── test
│   ├── images
│   │   ├── 0.jpg
│   │   ├── 10.jpg
│   │   ├── 11.jpg
│   │   ├── 12.jpg
│   │   ├── 13.jpg
│   │   ├── 14.jpg
│   │   ├── 15.jpg
│   │   ├── 16.jpg
│   │   ├── 17.jpg
│   │   ├── 18.jpg
│   │   ├── 19.jpg
│   │   ├── 1.jpg
│   │   ├── 20.jpg
│   │   ├── 21.jpg
│   │   ├── 22.jpg
│   │   ├── 23.jpg
│   │   ├── 24.jpg
│   │   ├── 25.jpg
│   │   ├── 26.jpg
│   │   ├── 27.jpg
│   │   ├── 28.jpg
│   │   ├── 29.jpg
│   │   ├── 2.jpg
│   │   ├── 30.jpg
│   │   ├── 31.jpg
│   │   ├── 32.jpg
│   │   ├── 33.jpg
│   │   ├── 34.jpg
│   │   ├── 3.jpg
│   │   ├── 4.jpg
│   │   ├── 5.jpg
│   │   ├── 6.jpg
│   │   ├── 7.jpg
│   │   ├── 8.jpg
│   │   └── 9.jpg
│   └── labels
│       ├── 0.txt
│       ├── 10.txt
│       ├── 11.txt
│       ├── 12.txt
│      

In [ ]:
ROOT_DIR  = '/content/drive/MyDrive/hackathon/datasets'
IMAGE_DIR = '/content/drive/MyDrive/hackathon/datasets/images/' # directory to save images
LABEL_DIR = '/content/drive/MyDrive/hackathon/datasets/labels/' # directory to save labels

# Добавить ссылки на dataframe 

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/hackathon/datasets/train.csv")
df

,image_id,width,height,bbox
0,0,7360,4912,"[4830, 4275, 66, 46]"
1,0,7360,4912,"[3537, 4646, 73, 23]"
2,1,7360,4912,"[447, 3073, 53, 142]"
3,10,7360,4912,"[359, 773, 79, 37]"
4,10,7360,4912,"[1845, 1207, 11, 13]"
...,...,...,...,...
4137,33_jpg.rf.4a981815d06750e3da15a6bd4e091c45,2048,1365,"[1312, 899, 3, 2]"
4138,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1650, 1086, 5, 4]"
4139,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1644, 1091, 5, 4]"
4140,31_jpg.rf.e1aa62bca610023c6d0983d14916cd43,2048,1365,"[255, 323, 6, 8]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4142 entries, 0 to 4141
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  4142 non-null   object
 1   width     4142 non-null   int64 
 2   height    4142 non-null   int64 
 3   bbox      4142 non-null   object
dtypes: int64(2), object(2)
memory usage: 129.6+ KB


In [ ]:
df['image_path'] = f'{IMAGE_DIR}' + df.image_id.apply(str) +'.jpg'
df['label_path'] = f'{LABEL_DIR}' + df.image_id.apply(str) + '.txt'
df['bbox'] = df['bbox'].progress_apply(eval)
df

  0%|          | 0/4142 [00:00<?, ?it/s]

,image_id,width,height,bbox,image_path,label_path
0,0,7360,4912,"[4830, 4275, 66, 46]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
1,0,7360,4912,"[3537, 4646, 73, 23]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
2,1,7360,4912,"[447, 3073, 53, 142]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
3,10,7360,4912,"[359, 773, 79, 37]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
4,10,7360,4912,"[1845, 1207, 11, 13]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
...,...,...,...,...,...,...
4137,33_jpg.rf.4a981815d06750e3da15a6bd4e091c45,2048,1365,"[1312, 899, 3, 2]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
4138,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1650, 1086, 5, 4]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
4139,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1644, 1091, 5, 4]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...
4140,31_jpg.rf.e1aa62bca610023c6d0983d14916cd43,2048,1365,"[255, 323, 6, 8]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...


In [ ]:
df.iloc[-1]["image_path"]

'/content/drive/MyDrive/hackathon/datasets/images/17_jpg.rf.0bfac2a737bdbf84975c255d9c70413a.jpg'

# Создать K-folds 

Неравное количество сэмплов в каждой папке может привести к большому расхождению результатов **Cross-Validation**

In [ ]:
from sklearn.model_selection import GroupKFold

kf = GroupKFold(n_splits = 5)
df = df.reset_index(drop=True)
df['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df, groups=df.image_id.tolist())):
    df.loc[val_idx, 'fold'] = fold

df.fold.value_counts()

1    829
0    829
4    828
2    828
3    828
Name: fold, dtype: int64

In [ ]:
df.head()

,image_id,width,height,bbox,image_path,label_path,fold
0,0,7360,4912,"[4830, 4275, 66, 46]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,2
1,0,7360,4912,"[3537, 4646, 73, 23]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,2
2,1,7360,4912,"[447, 3073, 53, 142]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,0
3,10,7360,4912,"[359, 773, 79, 37]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,0
4,10,7360,4912,"[1845, 1207, 11, 13]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,0


# Разделить набор данных на train и val

In [ ]:
FOLD = 1    # Выберите складку для тренировки

In [ ]:
train_files = []
val_files = []
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

(1577, 395)

In [ ]:
train_df.tail()

,image_id,width,height,bbox,image_path,label_path,fold
4137,33_jpg.rf.4a981815d06750e3da15a6bd4e091c45,2048,1365,"[1312, 899, 3, 2]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,2
4138,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1650, 1086, 5, 4]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,3
4139,34_jpg.rf.81e0639acdc8c56a68b4989a5646e7f5,2048,1366,"[1644, 1091, 5, 4]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,3
4140,31_jpg.rf.e1aa62bca610023c6d0983d14916cd43,2048,1365,"[255, 323, 6, 8]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,4
4141,17_jpg.rf.0bfac2a737bdbf84975c255d9c70413a,2048,1366,"[402, 412, 17, 22]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,4


In [ ]:
valid_df.head()

,image_id,width,height,bbox,image_path,label_path,fold
2,1,7360,4912,"[447, 3073, 53, 142]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,1
3,10,7360,4912,"[359, 773, 79, 37]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,1
4,10,7360,4912,"[1845, 1207, 11, 13]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,1
5,10,7360,4912,"[3824, 4003, 66, 58]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,1
17,104,6000,4000,"[1826, 3640, 41, 23]",/content/drive/MyDrive/hackathon/datasets/imag...,/content/drive/MyDrive/hackathon/datasets/labe...,1


# 🌟Создать файл конфигурации для YOLOv5 

Этот файл конфигурации помогает YOLOv5 узнать основные функции нашего набора данных. Это требует:

- Корневой каталог набора данных и относительный путь к каталогам изображений `train/val/test` (или файл `*.txt` с путями к изображениям)
- Количество классов в наборе данных через переменную `nc`
- Список имен классов: `['bear']`

In [ ]:
test_path = '/content/drive/MyDrive/hackathon/datasets/test/images'
os.path.join(test_path, os.listdir("/content/drive/MyDrive/hackathon/datasets/test/images")[0])

'/content/drive/MyDrive/hackathon/datasets/test/images/12.jpg'

In [ ]:
import yaml

cwd = '/content/'
test_path = '/content/drive/MyDrive/hackathon/datasets/test/images'

with open(os.path.join(cwd, 'train.txt'), 'w') as f:
    for path in train_df.image_path.unique().tolist():
        f.write(path + '\n')

with open(os.path.join(cwd, 'val.txt'), 'w') as f:
    for path in valid_df.image_path.unique().tolist():
        f.write(path + '\n')

with open(os.path.join(cwd, 'test.txt'), 'w') as f:
    for name in os.listdir(test_path):
        s = os.path.join(test_path, name)
        f.write(s + '\n')

data = dict(
    path = '/content/',
    train = os.path.join(cwd, 'train.txt'),
    val = os.path.join(cwd, 'val.txt'),
    test = os.path.join(cwd, 'test.txt'),
    nc = 1,
    names = ['bear'],
)

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- bear
nc: 1
path: /content/
test: /content/test.txt
train: /content/train.txt
val: /content/val.txt



In [ ]:
%%writefile /content/hyp2.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.8  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
copy_paste: 0.1  # segment copy-paste (probability)

Writing /content/hyp2.yaml


# Создать файл конфигурации для `YOLOv5l6`

In [ ]:
%%writefile /content/yolov5l6.yaml

# Parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [19,27,  44,40,  38,94]  # P3/8
  - [96,68,  86,152,  180,137]  # P4/16
  - [140,301,  303,264,  238,542]  # P5/32
  - [436,615,  739,380,  925,792]  # P6/64

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [768, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [768]],
   [-1, 1, Conv, [1024, 3, 2]],  # 9-P6/64
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 11
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [768, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 8], 1, Concat, [1]],  # cat backbone P5
   [-1, 3, C3, [768, False]],  # 15

   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 19

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 23 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 20], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 26 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 16], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [768, False]],  # 29 (P5/32-large)

   [-1, 1, Conv, [768, 3, 2]],
   [[-1, 12], 1, Concat, [1]],  # cat head P6
   [-1, 3, C3, [1024, False]],  # 32 (P6/64-xlarge)

   [[23, 26, 29, 32], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5, P6)
  ]

Writing /content/yolov5l6.yaml


# Download YOLOv5

In [ ]:
%cd /content/drive/MyDrive/hackathon/training/Yolov5

if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git


%cd yolov5
!pip install qr requirements.txt

import utils
display = utils.notebook_init()

YOLOv5 🚀 v6.2-159-gd669a74 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 37.6/166.8 GB disk)


# 🦽 Training

In [ ]:
DIM = 2100  # 3000x3000
MODEL = 'yolov5l6'
BATCH = 2
EPOCHS = 13

PROJECT   = 'polarbear' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

In [ ]:
!python train.py \
--img {DIM} \
--batch {BATCH} \
--epochs {EPOCHS} \
--data /content/gbr.yaml \
--hyp /content/hyp2.yaml \
--weights yolov5l6.pt \
--project {PROJECT} \
--name {NAME} \
--exist-ok

wandb: Currently logged in as: truongsown. Use `wandb login --relogin` to force relogin
train: weights=yolov5l6.pt, cfg=, data=/content/gbr.yaml, hyp=/content/hyp2.yaml, epochs=13, batch_size=2, imgsz=2100, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=polarbear, name=yolov5l6-dim2100-fold1, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.2-159-gd669a74 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs

In [ ]:
%cd /content/drive/MyDrive/hackathon/training/Yolov5/yolov5

/content/drive/MyDrive/hackathon/training/Yolov5/yolov5


In [ ]:
!rm -r /content/drive/MyDrive/hackathon/training/Yolov5/yolov5/runs

In [ ]:
!python detect.py --source /content/drive/MyDrive/hackathon/datasets/test/images/ --conf-thres 0.1 --iou-thres 0.2 --imgsz 1000 --weight /content/drive/MyDrive/hackathon/training/Yolov5/yolov5/polarbear/yolov5l6-dim2100-fold1/weights/best.pt

detect: weights=['/content/drive/MyDrive/hackathon/training/Yolov5/yolov5/polarbear/yolov5l6-dim2100-fold1/weights/best.pt'], source=/content/drive/MyDrive/hackathon/datasets/test/images/, data=data/coco128.yaml, imgsz=[1000, 1000], conf_thres=0.1, iou_thres=0.2, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-159-gd669a74 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 346 layers, 76118664 parameters, 0 gradients, 109.9 GFLOPs
WARNING ⚠️ --img-size [1000, 1000] must be multiple of max stride 64, updating to [1024, 1024]
image 1/39 /content/drive/MyDrive/hackathon/datasets/test/images/0.jpg: 704x1024 1 bear, 35.8ms
image 2/39 /content/drive/MyDriv